# PART 4 - Machine Learning models
#### top N labels, SVD et regression

In [17]:
from preprocessing import Preprocess, split_data
from utils import reconstruct_targets, inverse_pca

In [2]:
parquet_data_path = "../data/"
preprocess = Preprocess()
df = preprocess.create_dataframe(parquet_data_path, preprocess=True)

part-00004-1b8fcd71-6348-4510-a9dc-bdd7dcf82f2d-c000.snappy.parquet
part-00003-1b8fcd71-6348-4510-a9dc-bdd7dcf82f2d-c000.snappy.parquet
part-00001-1b8fcd71-6348-4510-a9dc-bdd7dcf82f2d-c000.snappy.parquet
part-00002-1b8fcd71-6348-4510-a9dc-bdd7dcf82f2d-c000.snappy.parquet
part-00000-1b8fcd71-6348-4510-a9dc-bdd7dcf82f2d-c000.snappy.parquet


In [3]:
df

,target,day,sous_domaine,domaine,top_domaine,tokens_path,100,1000,1001,1002,...,990,991,992,993,994,995,996,997,998,999
0,"[329, 1234, 5183, 96, 377]",4,www,societe,com,societ madam karin pinchon,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[158, 650, 1175, 831, 953]",4,www,ebay-kleinanzeigen,de,nu fbaumstamm,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[325, 253, 1775, 640, 543]",4,psychologie,aufeminin,com,forum copain troubl comport fd,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[1143, 210, 531, 18, 41]",4,fr.shopping,rakuten,com,poweron pr,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"[1171, 1071, 1192, 1533, 1277]",4,www,cdiscount,com,search coqu samsung galaxy,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67590,"[1193, 318, 22, 1107, 1111]",12,www,senscritique,com,ser hard critiqu,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67591,"[997, 1598, 993, 992, 984]",13,www,cdiscount,com,pret port vet blaz femm top manch longu vest f...,0,0,0,0,...,0,0,1,1,0,0,0,1,0,0
67592,"[1193, 1410, 1049, 1187, 358]",13,www,programme-tv,net,programm autr laventur robinson laventur robinson,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67593,"[63, 5184, 1254, 1119]",12,fr,windfinder,com,forecast point roug,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X_train, X_test, y_train, y_test, target_train, target_test = split_data(df, test_size=0.25)
X_train.shape

(50696, 6)
(16899, 6)
Index(['day', 'domaine', 'top_domaine', 'tokens_path'], dtype='object')
Index(['day', 'domaine', 'top_domaine', 'tokens_path'], dtype='object')


(50696, 9216)

In [5]:
X_train_multilabel = X_train
X_test_multilabel = X_test

In [7]:
reconstruct_targets(y_train, preprocess.mlb)[5]

[343, 365, 531, 705, 83]

### Multi-label classification
#### 1.  OneVsRest: 

In [ ]:
classifier1 = OneVsRestClassifier(LogisticRegression(penalty='l1', solver='liblinear', class_weight="balanced"), n_jobs=-1)
classifier1.fit(X_train_multilabel, y_train_multilabel)
predictions = classifier1.predict(X_test_multilabel)

In [37]:
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score

print("Accuracy :",metrics.accuracy_score(y_test_multilabel, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test_multilabel,predictions))
precision = precision_score(y_test_multilabel, predictions, average='micro')
recall = recall_score(y_test_multilabel, predictions, average='micro')
f1 = f1_score(y_test_multilabel, predictions, average='micro')
 
print("\nMicro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
precision = precision_score(y_test_multilabel, predictions, average='macro')
recall = recall_score(y_test_multilabel, predictions, average='macro')
f1 = f1_score(y_test_multilabel, predictions, average='macro')
 
print("\nMacro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
print("\nClassification Report")
print (metrics.classification_report(y_test_multilabel, predictions))

Accuracy : 0.007278537191549796
Hamming loss  0.0068070643314176215

Micro-average quality numbers
Precision: 0.2285, Recall: 0.7251, F1-measure: 0.3475

Macro-average quality numbers
Precision: 0.1337, Recall: 0.3929, F1-measure: 0.1884

Classification Report
              precision    recall  f1-score   support

           0       0.05      0.35      0.08        23
           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00         1
           3       0.10      0.20      0.13         5
           4       0.20      0.70      0.31        43
           5       0.09      0.33      0.14        18
           6       0.00      0.00      0.00         0
           7       0.07      0.23      0.10        22
           8       0.06      0.15      0.09        13
           9       0.03      0.25      0.06         4
          10       0.33      0.20      0.25         5
          11       0.12      0.54      0.19        52
          12       0.13      0.36   

## Partie 1

### Exploration: transformer le problème en un problème de regression après une réduction de dimension avec SVD
(je ne suis pas sur que cette technique soit efficace, car transformer le problème en regression n'est pas une bonne technique en général dans un problème de classification normal, cependant, le SVD permet de réduire la dimension d'une matrice sparse (nos labels) et donc peut s'averer interessant de tester cette méthode)

#### 1.MultiOutputRegressor
Cela suppose que les sorties sont indépendantes les unes des autres, ce qui pourrait ne pas être une hypothèse correcte pour nos labels et que l'enchainement séquentiel n'a pas d'importance

In [8]:
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR

In [13]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import random as sparse_random
n_comps = [30, 50, 200, 300, 350, 450]

for n in n_comps:
    svd = TruncatedSVD(n_components=n, random_state=42)
    svd.fit(y_train)
    print(svd.explained_variance_ratio_.sum())

0.3461522715829279
0.42954725909905467
0.6979537950184534
0.779875010567699
0.8104889781854889
0.8576244264618329


In [14]:
svd = TruncatedSVD(n_components=450, random_state=42)
svd.fit(y_train)
print(svd.explained_variance_ratio_.sum())

0.8576244264618329


In [21]:
y_train_svd= svd.transform(y_train)
y_test_svd = svd.transform(y_test)

In [24]:
y_train_svd_inversed = inverse_pca(y_train_svd, svd)
y_test_svd_inversed = inverse_pca(y_test_svd, svd)

(50696, 1903)
(16899, 1903)


In [25]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR
from sklearn.linear_model import Ridge
classifier1 = MultiOutputRegressor(Ridge(random_state=42), n_jobs=-1).fit(X_train, y_train_svd)

In [26]:
print("train score")
print(classifier1.score(X_train, y_train_svd))
print("test score")
print(classifier1.score(X_test, y_test_svd))


train score
0.3798330353205818
test score
0.17769464967793794


In [28]:
y_pred = inverse_pca(classifier1.predict(X_test), svd)

(16899, 1903)


In [29]:
### comparaison avec les vrais labels
from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred, average='weighted'))
print(f1_score(y_test, y_pred, average='macro'))

print(f1_score(y_test, y_pred, average='micro'))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.43041418222303723
0.12057983925316328
0.49684794841635854


In [30]:
#### n = 70
svd = TruncatedSVD(n_components=70, random_state=42)
svd.fit(y_train)
print(svd.explained_variance_ratio_.sum())

y_train_svd= svd.transform(y_train)
y_test_svd = svd.transform(y_test)

y_train_svd_inversed = inverse_pca(y_train_svd, svd)
y_test_svd_inversed = inverse_pca(y_test_svd, svd)

classifier1 = MultiOutputRegressor(Ridge(random_state=42), n_jobs=-1).fit(X_train, y_train_svd)


print("train score")
print(classifier1.score(X_train, y_train_svd))
print("test score")
print(classifier1.score(X_test, y_test_svd))


y_pred = inverse_pca(classifier1.predict(X_test), svd)

### comparaison avec les vrais labels
from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred, average='weighted'))
print(f1_score(y_test, y_pred, average='macro'))

print(f1_score(y_test, y_pred, average='micro'))

0.4887737876724469
(50696, 1903)
(16899, 1903)
train score
0.6033880415059729
test score
0.47153312441413514
(16899, 1903)


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.319132079491168
0.04937746962807044
0.414144742864466


#### 2.chained multioutput regression

In [31]:
from sklearn.multioutput import RegressorChain
# define base model
model_1 = LinearSVR()
# define the chained multioutput wrapper model
regressor_chain = RegressorChain(model_1).fit(X_train, y_train_svd)



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [33]:
classifier1 = regressor_chain
print("train score")
print(classifier1.score(X_train, y_train_svd))
print("test score")
print(classifier1.score(X_test, y_test_svd))


y_pred = inverse_pca(classifier1.predict(X_test), svd)

### comparaison avec les vrais labels
from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred, average='weighted'))
print(f1_score(y_test, y_pred, average='macro'))

print(f1_score(y_test, y_pred, average='micro'))

train score
0.5195245639186439
test score
0.4355396168555035
(16899, 1903)


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.319728440086272
0.050458625759732145
0.4138248598282727


## Partie 2 (ancienne version)
Il existe differentes approches pour filtrer les classes les moints utiles (pas forcemment celles avec le nombre d'occurences = 1).
(Cette approche est loin d'etre correcte pour notre problème, cependant j'ai voulu voir ce que ça pourrait donner)

#### Prendre les X labels les plus occurrents

In [91]:
categorical_features = ["day","domaine","top_domaine"]

tfidf = TfidfVectorizer(min_df=0)
vectorizer = TfidfVectorizer(min_df=0.00009, smooth_idf=True, norm="l2", tokenizer = lambda x: x.split(" "), sublinear_tf=False, ngram_range=(1,1))

transformer = ColumnTransformer([ ('categorical', OneHotEncoder(sparse=False, handle_unknown = "ignore"), categorical_features),
                            ("vectorizer", vectorizer, "tokens_path"),
                         ], remainder="passthrough")

X_train_multilabel = transformer.fit_transform(X_train)
X_test_multilabel = transformer.transform(X_test)
y_train_multilabel = y_train 
y_test_multilabel = y_test 

In [ ]:
#### 

#### 300 Labels

In [93]:
labels_vectorizer = CountVectorizer(binary='true', max_features=300).fit(y_train_inversed)


In [94]:
y_train_5labels = labels_vectorizer.transform(y_train_inversed)

In [104]:
y_test_5labels = labels_vectorizer.transform(y_test_inversed)

In [122]:
def display_score(y_test_5labels, predictions):
    print("Accuracy :",metrics.accuracy_score(y_test_5labels, predictions))
    print("Hamming loss ",metrics.hamming_loss(y_test_5labels,predictions))
    precision = precision_score(y_test_5labels, predictions, average='micro')
    recall = recall_score(y_test_5labels, predictions, average='micro')
    f1 = f1_score(y_test_5labels, predictions, average='micro')

    print("\nMicro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
    precision = precision_score(y_test_5labels, predictions, average='macro')
    recall = recall_score(y_test_5labels, predictions, average='macro')
    f1 = f1_score(y_test_5labels, predictions, average='macro')

    print("\nMacro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
    print("\nClassification Report")
    print (metrics.classification_report(y_test_5labels, predictions))

In [ ]:
#labels_vectorizer.inverse_transform(y_train_5labels)

In [ ]:
# comparer les résultats avec y_train et y_train_max_features

In [ ]:
classifier1 = OneVsRestClassifier(LogisticRegression(penalty='l1', solver='liblinear', class_weight="balanced"), n_jobs=-1)
classifier1.fit(X_train_multilabel, y_train_5labels)


In [106]:
predictions = classifier1.predict(X_test_multilabel) 

In [108]:
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score

print("Accuracy :",metrics.accuracy_score(y_test_5labels, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test_5labels,predictions))
precision = precision_score(y_test_5labels, predictions, average='micro')
recall = recall_score(y_test_5labels, predictions, average='micro')
f1 = f1_score(y_test_5labels, predictions, average='micro')
 
print("\nMicro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
precision = precision_score(y_test_5labels, predictions, average='macro')
recall = recall_score(y_test_5labels, predictions, average='macro')
f1 = f1_score(y_test_5labels, predictions, average='macro')
 
print("\nMacro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
print("\nClassification Report")
print (metrics.classification_report(y_test_5labels, predictions))

Accuracy : 0.03213207882123203
Hamming loss  0.022857762786752667

Micro-average quality numbers
Precision: 0.3017, Recall: 0.8126, F1-measure: 0.4400

Macro-average quality numbers
Precision: 0.2855, Recall: 0.7531, F1-measure: 0.3999

Classification Report
              precision    recall  f1-score   support

           0       0.30      0.79      0.44        78
           1       0.26      0.76      0.38       226
           2       0.52      0.87      0.65        89
           3       0.20      0.76      0.31        85
           4       0.21      0.59      0.31       126
           5       0.22      0.71      0.33        70
           6       0.10      0.57      0.17        93
           7       0.15      0.60      0.24       180
           8       0.17      0.64      0.26       152
           9       0.25      0.75      0.37       169
          10       0.23      0.91      0.36       163
          11       0.26      0.80      0.40        85
          12       0.34      0.81     

C:\Users\amine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#### Compare predictions et le vrai y_train

#### 30 features

In [124]:
predictions_train = classifier1.predict(X_train_multilabel)
display_score(y_train_5labels, predictions_train)

Accuracy : 0.5401609594445321
Hamming loss  0.027827310504444796

Micro-average quality numbers
Precision: 0.5580, Recall: 0.9957, F1-measure: 0.7152

Macro-average quality numbers
Precision: 0.5845, Recall: 0.9955, F1-measure: 0.7225

Classification Report
              precision    recall  f1-score   support

           0       0.50      1.00      0.67      1666
           1       0.59      1.00      0.74      1281
           2       0.84      1.00      0.91      1494
           3       0.45      0.99      0.62      1312
           4       0.62      0.99      0.77      2205
           5       0.93      1.00      0.96      1697
           6       0.79      1.00      0.88      2727
           7       0.50      1.00      0.66      1369
           8       0.52      1.00      0.69      1522
           9       0.75      1.00      0.86      2651
          10       0.53      0.99      0.69      1375
          11       0.62      1.00      0.76      1913
          12       0.62      0.99      

C:\Users\amine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [109]:
### 30 features
labels_vectorizer = CountVectorizer(binary='true', max_features=30).fit(y_train_inversed)
y_train_5labels = labels_vectorizer.transform(y_train_inversed)
y_test_5labels = labels_vectorizer.transform(y_test_inversed)

classifier1 = OneVsRestClassifier(LogisticRegression(penalty='l1', solver='liblinear', class_weight="balanced"), n_jobs=-1)
classifier1.fit(X_train_multilabel, y_train_5labels)
predictions = classifier1.predict(X_test_multilabel) 

In [123]:
display_score(y_test_5labels, predictions)

Accuracy : 0.480679330137878
Hamming loss  0.03218928141704638

Micro-average quality numbers
Precision: 0.5253, Recall: 0.9113, F1-measure: 0.6664

Macro-average quality numbers
Precision: 0.5517, Recall: 0.9041, F1-measure: 0.6723

Classification Report
              precision    recall  f1-score   support

           0       0.45      0.92      0.60       556
           1       0.53      0.89      0.67       428
           2       0.86      0.97      0.91       510
           3       0.39      0.85      0.54       409
           4       0.61      0.90      0.73       750
           5       0.95      0.97      0.96       563
           6       0.79      0.99      0.88       955
           7       0.47      0.93      0.63       433
           8       0.49      0.91      0.64       469
           9       0.74      0.99      0.85       916
          10       0.44      0.86      0.58       443
          11       0.58      0.95      0.73       618
          12       0.58      0.93      0.

C:\Users\amine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
classifier1 = OneVsRestClassifier(LogisticRegression(penalty='l1', solver='liblinear', class_weight="balanced"), n_jobs=-1)
classifier1.fit(X_train+_multilabel, y_train_5labels)
predictions = classifier1.predict(y_test_agglo) 